In [ ]:
!pip install langchain
!pip install openai
!pip install qdrant_client
!pip install datasets
!pip install tiktoken
!pip install langchain-community
!pip install PyMuPDF
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 20.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully uninstalled grpcio-1.64.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-datastore 2.19.0 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

0.New Cluster

In [ ]:
import os
from dotenv import load_dotenv
from qdrant_client import QdrantClient
import openai


# Load the .env file
load_dotenv('/content/drive/MyDrive/NASA Hunch - Psych AI/Hunch.env')

# Access the environment variables
qdrant_key = os.getenv('QDRANT_KEY')
qdrant_url = os.getenv('QDRANT_URL')
langchain_api_key = os.getenv('LANGCHAIN_API_KEY')
openai.api_key = os.getenv('OPENAI_API_KEY')

print(qdrant_key)
print(qdrant_url)
print(langchain_api_key)
print(openai.api_key)

# Example: Using the Qdrant client with the retrieved credentials
qdrant_client = QdrantClient(url=qdrant_url, api_key=qdrant_key)


In [ ]:
from qdrant_client.http import models

collection_config = models.VectorParams(
    size=384,  # dimensions of the vectors
    distance=models.Distance.COSINE
)

qdrant_client.create_collection(
    collection_name="Space Psychology",
    vectors_config=collection_config
)

UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `Space Psychology` already exists!"},"time":0.080076387}'

In [ ]:
# List all available collections
collections = qdrant_client.get_collections()
print(collections)

collections=[CollectionDescription(name='Space Psychology')]


Adding Data to RAG

In [ ]:
import os
import fitz  # PyMuPDF
from dotenv import load_dotenv
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

# Load environment variables
load_dotenv('.env')
qdrant_key = os.getenv('QDRANT_KEY')
qdrant_url = os.getenv('QDRANT_URL')

# Create Qdrant client
qdrant_client = QdrantClient(url=qdrant_url, api_key=qdrant_key)

# Load a pre-trained model for vectorization
model = SentenceTransformer('all-MiniLM-L6-v2')

# Function to extract text from a PDF file
def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as pdf:
        for page in pdf:
            text += page.get_text()
    return text

# Path to your PDF files
pdf_files = ["/content/THE BURDEN OF SPACE EXPLORATION ON THE MENTAL HEALTH OF ASTRONAUTS.pdf",
             '/content/Applications of methods of psychological support developed for astronauts for use in medical settings.pdf',
             '/content/Psychological Issues Relevant to Astronaut Selection for Long-Duration Space Flight_ A Review of the Literature.pdf',
             '/content/Space Missions Psychological Issues.pdf']  # Add paths to your PDF files

# Prepare data for Qdrant
points = []
for idx, pdf_file in enumerate(pdf_files):
    # Extract text from the PDF
    text = extract_text_from_pdf(pdf_file)

    # Generate vector representation
    vector = model.encode(text).tolist()  # Convert to a list for Qdrant

    # Create a point for Qdrant with a unique integer ID
    point = {
        'id': idx,  # Use an integer ID
        'vector': vector,
        'payload': {
            'text': text,  # Store the extracted text
            'pdf_file': pdf_file,  # Store the file path or name
        }
    }
    points.append(point)

# Upsert the vectors into Qdrant
qdrant_client.upsert(collection_name="Space Psychology", points=points)

print("Research papers have been added to the collection.")


Research papers have been added to the collection.


In [ ]:
query_vector = model.encode("mood assessment in space")  # Example query
results = qdrant_client.search(
    collection_name="Space Psychology",
    query_vector=query_vector,
    limit=5  # Number of results to return
)
print(results)

[ScoredPoint(id=3, version=1, score=0.5756447, payload={'text': 'Space missions: psychological and\npsychopathological issues\nDonatella Marazziti1,2*\n, Alessandro Arone1, Tea Ivaldi1, Konstantin Kuts3 and\nKonstantin Loganovsky3\n1Department of Clinical and Experimental Medicine University of Pisa, Pisa, Italy, 2Unicamillus-Saint Camillus\nInternational University of Medical and Health Sciences, Rome, Italy, and 3Department of Radiation Psychoneurol-\nogy, Institute for Clinical Radiology, State Institution, National Research Centre for Radiation Medicine, National\nAcademy of Medical Sciences of Ukraine, Kyiv, Ukraine\nAbstract\nExploring space is one of the most attractive goals that humanity ever set, notwithstanding, there\nare some psychological and psychopathological risks that should be considered. Several studies\nidentified some possible hazards of space travels and related physical and psychological\nconsequences on astronauts. If some psychological reactions are obviously 

Linking to LLM

In [ ]:
import re

def clean_document(text):
    # Remove references section based on common patterns
    cleaned_text = re.sub(r'(References?|Bibliography).*?$', '', text, flags=re.DOTALL)
    return cleaned_text.strip()

In [ ]:
def extract_relevant_sections(text):
    sections = text.split("\n\n")  # Split by paragraphs or any other delimiter
    relevant_sections = []

    for section in sections:
        if "Introduction" in section or "Conclusion" in section or "Abstract" in section:  # Keep only specific sections
            relevant_sections.append(section)

    return "\n\n".join(relevant_sections)

In [ ]:
def retrieve_relevant_documents(query, collection_name="Space Psychology", top_k=1):
    # Encode the query using the same model used for indexing
    query_vector = model.encode(query).tolist()

    # Perform a search in Qdrant
    search_result = qdrant_client.search(
        collection_name=collection_name,
        query_vector=query_vector,
        limit=top_k  # Retrieve top K most relevant documents
    )

    # Extract the text from the retrieved documents and clean them
    retrieved_texts = [clean_document(hit.payload['text']) for hit in search_result]
    return retrieved_texts

In [ ]:
def generate_response_with_context(user_query, retrieved_texts):
    # Combine the retrieved documents into context
    context = "\n\n".join(retrieved_texts)

    # Create a chat-based prompt
    messages = [
        {"role": "system", "content": "You are an expert on astronauts' mental health."},
        {"role": "assistant", "content": f"The following context is from research papers:\n{context}"},
        {"role": "user", "content": user_query}
    ]

    # Generate a response using the Chat API
    response = openai.chat.completions.create(
        model="gpt-4",  # You can use 'gpt-3.5-turbo' or 'gpt-4'
        messages=messages,
        max_tokens=500,  # Adjust the token limit based on your needs
        temperature=0.7
    )

    # Access the content of the first choice correctly
    return response.choices[0].message.content  # Corrected access to content

In [ ]:
import openai
from qdrant_client import QdrantClient
from qdrant_client.http import models
from sentence_transformers import SentenceTransformer
import os

# Load environment variables for API keys
openai.api_key = os.getenv('OPENAI_API_KEY')  # Your GPT API key

# Initialize Qdrant Client with the URL and key
qdrant_client = QdrantClient(
    url=os.getenv('QDRANT_URL'),
    api_key=os.getenv('QDRANT_KEY')
)

# Load the model used for creating query vectors (same one used for indexing)
model = SentenceTransformer('all-MiniLM-L6-v2')


# Example usage
user_query = "I have been feeling down, what should I do?"

# Retrieve relevant documents from Qdrant
retrieved_documents = retrieve_relevant_documents(user_query)

# Generate a response using the retrieved context
final_response = generate_response_with_context(user_query, retrieved_documents)

print("Response from the model:")
print(final_response)

Response from the model:
Astronauts face several psychological challenges during long-duration space missions, which can potentially lead to psychological problems. 

1. Physical Factors: Microgravity can impact cell structure and differentiation, as well as the immune and central nervous system. The central nervous system needs to adapt to microgravity since different somatosensory, visual, and vestibular information need to be processed. Space radiation also represents a risk during space missions, leading to several changes in the central nervous system, like shifts in brain fluids, altered sensory perception, and neurovestibular problems.

2. Habitability: Factors of "habitability", referring to the main characteristics of the spacecraft, can contribute to the overall well-being of astronauts. These include light, noise, vibration, and temperature. 

3. Individual Issues: Isolation from family, friends, and life on Earth, coupled with hard work continuously monitored, can lead to p